## For Colab only
We need to clone the repo here before we can use it

In [ ]:
!git init
!git remote add origin "https://github.com/karan-sarkar/VGGVox-PyTorch.git" 
!git pull origin master
!git branch --set-upstream-to=origin/master master

In [ ]:
!pip install -r requirements.txt

Download dataset from gdrive. The specified file is just a minimal copy of the dataset. You can upload a different file and change the ID to your file.

In [ ]:
!gdown --id '19q3G3XWKCkjwzdAWf8E-Ntx_hG-shRxo' -O data/wav.zip
!unzip ./data/wav.zip -d ./data/ && mv ./data/subset ./data/wav

In [ ]:
data_dir = "./data"

## Start

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import torch
sys.path.append(os.path.abspath('..'))
from train_fsl import Experiment, get_model

In [3]:
try: data_dir
except: data_dir = "/Users/pi/repos/gatech/8803-LS/Project/dataset/voxceleb2"


NameError: name 'dataloaders' is not defined

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
experiment = Experiment(batch_size=1, num_workers=0, device=device)

In [6]:
loaders = experiment.dataloaders(data_dir)
batch = next(iter(loaders['train']))

In [7]:
batch[0].shape

torch.Size([25, 1, 512, 300])

In [9]:
from audionet import AudioNet
from easyfsl.methods import RelationNetworks
N_WAY = 5 
model = get_model(pretrained=False, backbone_arch='audionet', num_ways=N_WAY)
model = model.to(device)

In [ ]:
experiment.train(model, loaders)

Start Training


Epoch [1/1]:   0%|                                                                                                                                                       | 0/100 [00:00<?, ?it/s]